In [2]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import models

Using TensorFlow backend.


In [ ]:
# Readinging SMILES strings from JSON file

raw_dataframe = 'C:/Users/koolk/Downloads/m1507656/df_62k.json'
df_62k = pd.read_json(raw_dataframe, orient='split')


molecules = df_62k['canonical_smiles'].values
maxlen = len(max(molecules, key=len))

molecules = molecules.reshape(-1, 1)
molecules = molecules.astype(str)

molecules = np.char.zfill(molecules, width=maxlen)

In [81]:
test = molecules[0:1000, :]
test1 = test.tolist()
test2 = [list(x[0]) for x in test1]

In [82]:
def encoded_smiles(string_array):
    '''
    Convert SMILES Strings into OneHotEncode 2D arrays
    
    Inputs
    -----
    
    Returns
    -------
    
    Examples
    --------
    
    
    '''
    
    enc = OneHotEncoder(handle_unknown='ignore')
    x1 = enc.fit(string_array)
    x1 = x1.transform(string_array).toarray()

    return x1

In [86]:
x1 = encoded_smiles(test2)

In [87]:
x1.shape

(1000, 1981)

In [96]:
x1[0, 500:900] - x1[3, 500:900]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0

In [95]:
x1[3, 500:900]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [35]:
test = [['c', 'C', 'n'],['C', '1', '('],['t','1','0']]

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(test)
x1 = enc.transform(test).toarray()

In [39]:
enc.transform([['c', 'c', 'c']]).toarray()

array([[0., 1., 0., 0., 0., 0., 0., 0.]])

In [41]:
enc.inverse_transform(x1)

array([['c', 'C', 'n'],
       ['C', '1', '('],
       ['t', '1', '0']], dtype=object)

In [76]:
test1 = test.tolist()
test2 = [list(x[0]) for x in test1]

In [79]:
x1 = encoded_smiles(test2)

In [80]:
x1.shape

(10, 594)